# how to use my model

Download the model weights from GitHub:

In [1]:
!curl -OL https://github.com/cmhainje/ml4phys-hw/raw/main/connor_net.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  137k  100  137k    0     0  1514k      0 --:--:-- --:--:-- --:--:-- 1514k


Load the model by first making the network, and then loading the `state_dict` from the `.pt` file.

In [2]:
import torch
import torch.nn as nn

# Use a GPU if you have one!
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # use Metal if on M-series MacBook
else:
    device = torch.device("cpu")
    
# Model definition:
kw = dict(kernel_size=3, padding='same', padding_mode='circular')
net = nn.Sequential(
    nn.Conv1d(1, 4, **kw),
    nn.ReLU(),
    nn.Conv1d(4, 4, **kw),
    nn.ReLU(),
    nn.Conv1d(4, 4, **kw),
    nn.ReLU(),
    nn.Conv1d(4, 4, **kw),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(4 * 8575, 1)
).to(device)
state_dict = torch.load('connor_net.pt', map_location=device)
net.load_state_dict(state_dict)

<All keys matched successfully>

Now that it's loaded, here's how to use it on data.

You **must** call `.unsqueeze(1)` on the input so that it has the right shape for the model (the model expects shape `(N, 1, 8575)`).

the model then returns a tensor of shape `(N, 1)`, so you can call `.squeeze()` to make it shape `(N,)`.

In [3]:
import numpy as np
X_te = torch.tensor(np.load('hw3_data/test_features.npy')).to(torch.float32).to(device)
print('Test data shape: ', X_te.shape)

out = net(X_te.unsqueeze(1))
print('Net output shape:', out.shape)
out = out.squeeze()
print('After squeezing: ', out.shape)

Test data shape:  torch.Size([512, 8575])
Net output shape: torch.Size([512, 1])
After squeezing:  torch.Size([512])
